In [ ]:
%pip install llama-index-llms-openai
!pip install llama-index

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

import nest_asyncio
nest_asyncio.apply()


In [14]:
from llama_index.core import (
    PromptTemplate,
)


In [ ]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("open ai api key: ")

In [5]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")

In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
# documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()

In [ ]:
from llama_index.core import VectorStoreIndex

# vector_index = VectorStoreIndex.from_documents(documents, embed_model=ollama_embedding)
vector_index = VectorStoreIndex.from_documents(documents)


In [6]:
def format_context_fn(**kwargs):
    # Extract and split the context into paragraphs
    context = kwargs["context_str"]
    paragraphs = context.split("\n\n")
    # Format each paragraph as a bullet point
    formatted_context = "\n\n".join([f"- {p.strip()}" for p in paragraphs if p.strip()])
    return formatted_context


In [10]:
qa_prompt_tmpl_str = """\
Context information is below:
---------------------
{context_str}
---------------------
Question: {query_str}
Answer: \
"""

prompt_tmpl = PromptTemplate(
    qa_prompt_tmpl_str,
    function_mappings={"context_str": format_context_fn}
)


In [ ]:
# Extract the first 1000 characters from the document as the context
context_str = documents[0].get_text()[:1000]
query_str = "What are the main ideas in this essay?"

formatted_prompt = prompt_tmpl.format(
    context_str=context_str,  # Pass the dynamic context to the formatter
    query_str=query_str
)

print("\nFormatted Prompt:")
print(formatted_prompt)


In [ ]:
response = vector_index.as_query_engine(
    llm=ollama_llm
).query(formatted_prompt)

print("\nResponse from the Query Engine:")
print(response)
